# Introduction
This code implements the Mahdiable algorithm to calculate the profit if an exchange arbtrage is executed on a given cryptocurrency.  
Inputs:
- Limit order book on the source exchange
- Limit order book on the destination exchange
- Commission or transaction fee on both sides
- Base amount of withdraw fee on the souce exchange  

Outputs
for every given symbol (product pair such as BTC-USD) it returns the following values:
- profit: The profit can be gain by the arbitrage
- trading_volume: The optimum base amount that should be bought at the source exchange
- spent_money: The total amount of quote (money) should be spent
- sold_money: The total amount of quote (money) this pair can be sold

# Functions and data structures
## Stack Data Structure

In [9]:
class StackLob:
    def __init__(self):
        self.items = []

    def __repr__(self):
        return f"Stack({self.items})"

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.is_empty():
            return self.items.pop()

    def peek(self):
        if not self.is_empty():
            return self.items[-1]

    def is_empty(self):
        return len(self.items) == 0

    def size(self):
        return len(self.items)

    def process_order_book_to_stack(self, order_book_df, price_col, volume_col, is_sell_side=True):
        df = order_book_df[[price_col, volume_col]].copy()
        ascending_sort_order = not is_sell_side #'ascending' if is_sell_side else 'descending'
        sorted_df = df.sort_values(by=price_col, ascending=ascending_sort_order)
        orders = sorted_df.to_dict('records')  # Convert rows to dictionaries
        for order in orders:
            self.push(order)
        return self


## Withdraw handling Function

In [10]:
def update_withdrawal_fee_money(src_price,
                                current_trading_volume,
                                withdrawal_fee_quote_amount,
                                remaining_withdrawal_fee_amount):
    if remaining_withdrawal_fee_amount > 0:
        w_volume = min(current_trading_volume, remaining_withdrawal_fee_amount)
        withdrawal_fee_quote_amount += src_price * w_volume
        remaining_withdrawal_fee_amount -= w_volume
        return withdrawal_fee_quote_amount, remaining_withdrawal_fee_amount
    else:
        return withdrawal_fee_quote_amount, remaining_withdrawal_fee_amount


## Calculate Profits based on spent money and earned money

In [11]:
def calculate_profit(spent_money,
                     sold_money,
                     withdrawal_fee_quote_amount,
                     src_transaction_fee_rate,
                     dst_transaction_fee_rate, ):
    total_buy_cost = spent_money * (1 + src_transaction_fee_rate)

    total_sell_gain = sold_money * (1 - dst_transaction_fee_rate) - withdrawal_fee_quote_amount
    profit = total_sell_gain - total_buy_cost
    return profit, total_buy_cost, total_sell_gain


In [12]:
# Mahdiable Algorithm

In [13]:
def mahdiable_algorithm(src_sell_order_book_df,
                        dst_buy_order_book_df,
                        price_col,
                        volume_col,
                        withdrawal_fee_base_amount):
    src_sell_stack = StackLob().process_order_book_to_stack(src_sell_order_book_df, price_col,
                                                            volume_col, is_sell_side=True)
    dst_buy_stack = StackLob().process_order_book_to_stack(dst_buy_order_book_df, price_col,
                                                           volume_col, is_sell_side=False)
    spent_money = 0
    cum_traded_volume = 0
    sold_money = 0
    rem_w_fee_amount = withdrawal_fee_base_amount
    withdrawal_fee_money = 0

    while not src_sell_stack.is_empty() and not dst_buy_stack.is_empty():
        src_order = src_sell_stack.pop()
        dst_order = dst_buy_stack.pop()

        # [The main operation]
        if dst_order[price_col] <= src_order[price_col]:
            break

        # Calculate the min of possible trading volume
        current_trading_volume = min(src_order[volume_col], dst_order[volume_col])

        withdrawal_fee_money, rem_w_fee_amount = update_withdrawal_fee_money(src_order[price_col],
                                                                             current_trading_volume,
                                                                             withdrawal_fee_money,
                                                                             rem_w_fee_amount)

        # Calculate amount bought and amount to be sold
        cum_traded_volume += current_trading_volume
        # Update the volumes in the orders
        src_order[volume_col] -= current_trading_volume
        dst_order[volume_col] -= current_trading_volume

        spent_money += current_trading_volume * src_order[price_col]
        sold_money += current_trading_volume * dst_order[price_col]

        # If there's remaining volume, put the order back to the stack
        if src_order[volume_col] > 0:
            src_sell_stack.push(src_order)
        if dst_order[volume_col] > 0:
            dst_buy_stack.push(dst_order)

    return cum_traded_volume, spent_money, sold_money, withdrawal_fee_money


# Calculate the Symbol Profit

In [14]:
def calculate_symbol_profit_mahiable(order_book_df,
                                     withdrawal_fee_quote_amount,
                                     src_exchange_name,
                                     dst_exchange_name,
                                     src_transaction_fee_rate=0.001,
                                     dst_transaction_fee_rate=0.001,
                                     price_col='price',
                                     volume_col='volume',
                                     side_col='side',
                                     exchange_name_col='exchange_name',
                                     profit_col='profit',
                                     spent_money_col='spent_money',
                                     sold_money_col='sold_money',
                                     trading_volume_col='trading_volume'):
    order_book_src_df = order_book_df[order_book_df[exchange_name_col] == src_exchange_name]
    order_book_src_df_sell = order_book_src_df[order_book_src_df[side_col] == 'sell']

    order_book_dst_df = order_book_df[order_book_df[exchange_name_col] == dst_exchange_name]
    order_book_dst_df_buy = order_book_dst_df[order_book_dst_df[side_col] == 'buy']

    cum_traded_volume, spent_money, sold_money, withdrawal_fee_quote_amount = \
        mahdiable_algorithm(src_sell_order_book_df=order_book_src_df_sell,
                            dst_buy_order_book_df=order_book_dst_df_buy,
                            price_col=price_col,
                            volume_col=volume_col,
                            withdrawal_fee_base_amount=withdrawal_fee_quote_amount)

    profit, total_buy_cost, total_sell_gain = \
        calculate_profit(spent_money=spent_money,
                         sold_money=sold_money,
                         withdrawal_fee_quote_amount=withdrawal_fee_quote_amount,
                         src_transaction_fee_rate=src_transaction_fee_rate,
                         dst_transaction_fee_rate=dst_transaction_fee_rate)
    return {profit_col: profit,
            spent_money_col: total_buy_cost,
            sold_money_col: total_sell_gain,
            trading_volume_col: cum_traded_volume}


# Tests

In [17]:
import pandas as pd

src_exchange_name = 'Exchange1'
dst_exchange_name = 'Exchange2'
price_col = 'price'
volume_col = 'volume'
side_col = 'side'
exchange_name_col = 'exchange_name'
symbol = 'BTC'
order_book_rows = [
    # src exchange:
    (86, 5, symbol, 'buy', src_exchange_name),
    (97, 1, symbol, 'buy', src_exchange_name),
    (100, 2, symbol, 'sell', src_exchange_name),
    (102, 5, symbol, 'sell', src_exchange_name),
    (110, 6, symbol, 'sell', src_exchange_name),
    # dst exchange:
    (101, 5, symbol, 'buy', dst_exchange_name),
    (104, 2.5, symbol, 'buy', dst_exchange_name),
    (106, 1, symbol, 'buy', dst_exchange_name),
    (111, 0.5, symbol, 'sell', dst_exchange_name),
    (125, 2, symbol, 'sell', dst_exchange_name)
]

order_book_df = pd.DataFrame(order_book_rows,
                             columns=[price_col, volume_col, 'symbol',
                                      side_col, exchange_name_col])

withdrawal_fee = 0.1  # Sample withdrawal fee

src_order_book_df = order_book_df[order_book_df[exchange_name_col] == src_exchange_name]
src_sell_order_book_df = src_order_book_df[src_order_book_df[side_col] == 'sell']

dst_order_book_df = order_book_df[order_book_df[exchange_name_col] == dst_exchange_name]
dst_buy_order_book_df = dst_order_book_df[dst_order_book_df[side_col] == 'buy']


def test_compute_mahidable():
    result = mahdiable_algorithm(src_sell_order_book_df,
                                 dst_buy_order_book_df,
                                 price_col=price_col,
                                 volume_col=volume_col,
                                 withdrawal_fee_base_amount=withdrawal_fee)
    print("The results of Mahdiable algorithm on the test limit order book:" , result)
    assert (3.5, 353.0, 366.0) == result[:3]
    assert 10 == result[3]


def test_calculate_profit():
    profit_col = 'profit',
    spent_money_col = 'spent_money',
    sold_money_col = 'sold_money',
    trading_volume_col = 'trading_volume'
    transaction_fee_rate = 0.001
    result = calculate_symbol_profit_mahiable(order_book_df,
                                              withdrawal_fee,
                                              src_exchange_name,
                                              dst_exchange_name,
                                              src_transaction_fee_rate=transaction_fee_rate,
                                              dst_transaction_fee_rate=transaction_fee_rate,
                                              price_col=price_col,
                                              volume_col=volume_col,
                                              side_col=side_col,
                                              exchange_name_col=exchange_name_col,
                                              profit_col=profit_col,
                                              spent_money_col=spent_money_col,
                                              sold_money_col=sold_money_col,
                                              trading_volume_col=trading_volume_col)
    cum_traded_volume, spent_money, sold_money = (
        3.5, 353.0, 366.0)  # Actual values coming from compute_trade_volume_and_money
    direct_results = calculate_profit(spent_money=spent_money,
                                      sold_money=sold_money,
                                      withdrawal_fee_quote_amount=10,
                                      src_transaction_fee_rate=transaction_fee_rate,
                                      dst_transaction_fee_rate=transaction_fee_rate)
    assert result[profit_col] == direct_results[0]
    assert result[spent_money_col] == direct_results[1]
    assert result[sold_money_col] == direct_results[2]


In [18]:
test_compute_mahidable()
test_calculate_profit()

The results of Mahdiable algorithm on the test limit order book: (3.5, 353.0, 366.0, 10.0)
